In [121]:
import pandas as pd
from datetime import timedelta
import numpy as np
import time
import datetime
from matplotlib import pyplot
import warnings
warnings.filterwarnings("ignore")

In [122]:
trials = pd.read_csv('prediction_trials.tsv', header=0, sep='\t')
eventsDF = pd.read_csv("events_train.tsv", sep="\t",header=0)

In [ ]:
eventsDF

In [10]:
tlist = []
for i in range(len(trials)):
    tlist.append(trials.loc[i])

In [ ]:
tlist[0]

In [152]:
events = eventsDF[['closed_tstamp','event_type','latitude','longitude']]
events["year"] = pd.to_numeric(events["closed_tstamp"].str[:4], errors='coerce')

In [153]:
eventTypes = ['accidentsAndIncidents','roadwork', 'precipitation', 'deviceStatus', 'obstruction', 'trafficConditions']

In [154]:
trialEvents = pd.DataFrame(index=range(len(trials)))

In [155]:
mypoint = events[[ 'event_type', 'year', 'longitude', 'latitude']]
mypoint = mypoint.as_matrix()

In [156]:
pts = mypoint[:, [2, 3]]

In [286]:
start_time = time.time()

ss = []

for tin in range(2):
    
    edict = {}

    x1, x2 = trials.loc[tin]['nw_lon'], trials.loc[tin]['se_lon']
    y1, y2 = trials.loc[tin]['se_lat'], trials.loc[tin]['nw_lat']


    ll = np.array([x1, y1])  # lower-left
    ur = np.array([x2, y2])  # upper-right

    inidx = np.all(np.logical_and(ll <= pts, pts <= ur), axis=1)
    #inbox = pts[inidx]
    pos = np.where(inidx==True)
    
    xx = pd.DataFrame(mypoint[pos]).groupby([0, 1]).count().reset_index()
    for group in eventTypes:
        yy = xx[xx[0] == group].drop([0,3],1)
        z1 = xx[xx[0] == group].drop([0,3],1)[1].tolist()
        z2 = xx[xx[0] == group].drop([0,3],1)[2].tolist()
        edict[group] = [z1,z2]
    
    aa = pd.DataFrame.from_dict(edict, orient='index').as_matrix().tolist()
    ss.append(aa)    
    
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.220999956131 seconds ---


In [287]:
len(aa)

6

In [160]:
import pickle

In [161]:
pickle.dump(ss, fh)

In [162]:
fh = open('note.txt', 'r')

In [163]:
inss = pickle.load(fh)


In [265]:
inss[0]

{'accidentsAndIncidents': [[2003.0,
   2004.0,
   2005.0,
   2006.0,
   2007.0,
   2008.0,
   2009.0,
   2010.0,
   2011.0,
   2012.0,
   2013.0,
   2014.0],
  [1, 18, 22, 31, 47, 40, 114, 161, 278, 513, 467, 279]],
 'deviceStatus': [[2011.0, 2012.0, 2013.0, 2014.0], [3, 5, 7, 9]],
 'obstruction': [[2005.0,
   2007.0,
   2008.0,
   2009.0,
   2010.0,
   2011.0,
   2012.0,
   2013.0,
   2014.0],
  [1, 7, 3, 11, 6, 10, 17, 23, 15]],
 'precipitation': [[2011.0], [1]],
 'roadwork': [[2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0],
  [9, 3, 8, 9, 30, 27, 33, 1]],
 'trafficConditions': [[2006.0, 2007.0, 2009.0], [1, 1, 3]]}

In [270]:
pd.DataFrame.from_dict(inss[0], orient='index')

,0,1
accidentsAndIncidents,"[2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008....","[1, 18, 22, 31, 47, 40, 114, 161, 278, 513, 46..."
deviceStatus,"[2011.0, 2012.0, 2013.0, 2014.0]","[3, 5, 7, 9]"
obstruction,"[2005.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011....","[1, 7, 3, 11, 6, 10, 17, 23, 15]"
trafficConditions,"[2006.0, 2007.0, 2009.0]","[1, 1, 3]"
roadwork,"[2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013....","[9, 3, 8, 9, 30, 27, 33, 1]"
precipitation,[2011.0],[1]


In [262]:
aa = pd.DataFrame.from_dict(inss[0], orient='index').as_matrix()

In [263]:
aa

array([[ [2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0],
        [1, 18, 22, 31, 47, 40, 114, 161, 278, 513, 467, 279]],
       [[2011.0, 2012.0, 2013.0, 2014.0], [3, 5, 7, 9]],
       [[2005.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0],
        [1, 7, 3, 11, 6, 10, 17, 23, 15]],
       [[2006.0, 2007.0, 2009.0], [1, 1, 3]],
       [[2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0],
        [9, 3, 8, 9, 30, 27, 33, 1]],
       [[2011.0], [1]]], dtype=object)

In [245]:
c = []
for i in aa:
    c.extend(i.tolist())
print c

[[2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0], [1, 18, 22, 31, 47, 40, 114, 161, 278, 513, 467, 279], [2011.0, 2012.0, 2013.0, 2014.0], [3, 5, 7, 9], [2005.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0], [1, 7, 3, 11, 6, 10, 17, 23, 15], [2006.0, 2007.0, 2009.0], [1, 1, 3], [2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0], [9, 3, 8, 9, 30, 27, 33, 1], [2011.0], [1]]


In [249]:
c[3]

[3, 5, 7, 9]

In [232]:
pd.DataFrame.transpose(pd.DataFrame(aa.flatten().tolist()))

,0,1,2,3,4,5,6,7,8,9,10,11
0,2003.0,1.0,2011.0,3.0,2005.0,1.0,2006.0,1.0,2008.0,9.0,2011.0,1.0
1,2004.0,18.0,2012.0,5.0,2007.0,7.0,2007.0,1.0,2009.0,3.0,NaN,NaN
2,2005.0,22.0,2013.0,7.0,2008.0,3.0,2009.0,3.0,2010.0,8.0,NaN,NaN
3,2006.0,31.0,2014.0,9.0,2009.0,11.0,NaN,NaN,2011.0,9.0,NaN,NaN
4,2007.0,47.0,NaN,NaN,2010.0,6.0,NaN,NaN,2012.0,30.0,NaN,NaN
5,2008.0,40.0,NaN,NaN,2011.0,10.0,NaN,NaN,2013.0,27.0,NaN,NaN
6,2009.0,114.0,NaN,NaN,2012.0,17.0,NaN,NaN,2014.0,33.0,NaN,NaN
7,2010.0,161.0,NaN,NaN,2013.0,23.0,NaN,NaN,2015.0,1.0,NaN,NaN
8,2011.0,278.0,NaN,NaN,2014.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN
9,2012.0,513.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [240]:
c = []
for i in aa:
    c.extend(i.tolist())
print c

[[2003.0, 2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0], [1, 18, 22, 31, 47, 40, 114, 161, 278, 513, 467, 279], [2011.0, 2012.0, 2013.0, 2014.0], [3, 5, 7, 9], [2005.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0], [1, 7, 3, 11, 6, 10, 17, 23, 15], [2006.0, 2007.0, 2009.0], [1, 1, 3], [2008.0, 2009.0, 2010.0, 2011.0, 2012.0, 2013.0, 2014.0, 2015.0], [9, 3, 8, 9, 30, 27, 33, 1], [2011.0], [1]]


In [243]:
len(c)

12

In [88]:
np.array(pd.DataFrame.from_dict(inss[0], orient='index')[])

TypeError: data type "<i11" not understood

In [179]:
inss[1]

{'accidentsAndIncidents': [[2004.0,
   2005.0,
   2006.0,
   2007.0,
   2009.0,
   2010.0,
   2011.0,
   2012.0,
   2013.0,
   2014.0],
  [2, 3, 2, 2, 13, 10, 9, 11, 10, 12]],
 'deviceStatus': [[2013.0, 2014.0], [5, 3]],
 'obstruction': [[2011.0, 2012.0, 2013.0, 2014.0], [4, 6, 5, 4]],
 'precipitation': [[], []],
 'roadwork': [[2010.0, 2011.0, 2012.0, 2013.0, 2014.0], [1, 7, 2, 4, 21]],
 'trafficConditions': [[], []]}

In [30]:
type(inss[0])

dict

In [37]:
mypoint[pos]

array([['trafficConditions', '2003', -76.7720078, 38.943014],
       ['accidentsAndIncidents', '2003', -76.7634218, 38.94520429999999],
       ['accidentsAndIncidents', '2004', -76.7695091, 38.9448492],
       ..., 
       ['accidentsAndIncidents', '2014', -76.770179, 38.94529],
       ['accidentsAndIncidents', '2014', -76.770179, 38.94529],
       ['obstruction', '2014', -76.770179, 38.94529]], dtype=object)

In [36]:
xx = pd.DataFrame(mypoint[pos]).groupby([0, 1]).count().reset_index()
xx[xx[0] == group]

,0,1,2,3
28,trafficConditions,2003,1,1
29,trafficConditions,2009,1,1


In [47]:
xx = pd.DataFrame(mypoint[pos]).groupby([0, 1]).count().reset_index()
#for group in eventTypes:
yy = xx[xx[0] == group].drop([0,3],1)
zz = list(yy.itertuples(index=False))
print group, zz

trafficConditions [Pandas(_0='2003', _1=1), Pandas(_0='2009', _1=1)]


In [11]:
xx

,2,3,0,1
0,accidentsAndIncidents,2003,1,1
1,accidentsAndIncidents,2004,4,4
2,accidentsAndIncidents,2005,5,5
3,accidentsAndIncidents,2006,1,1
4,accidentsAndIncidents,2007,3,3
5,accidentsAndIncidents,2008,40,40
6,accidentsAndIncidents,2009,83,83
7,accidentsAndIncidents,2010,145,145
8,accidentsAndIncidents,2011,149,149
9,accidentsAndIncidents,2012,119,119


In [15]:
yy = xx[xx[2] == 'accidentsAndIncidents'].drop([1,2],1)
yy
list(yy.itertuples(index=False))

,3,0
0,2003,1
1,2004,4
2,2005,5
3,2006,1
4,2007,3
5,2008,40
6,2009,83
7,2010,145
8,2011,149
9,2012,119


In [22]:
zz = list(yy.itertuples(index=False))

In [24]:
len(zz)

12

In [ ]:
xx = events.groupby(['event_type', 'year']).count().reset_index()
xx[xx['event_type'] == 'DelayStatusCancellation'].groupby('year').count().reset_index()


list(df.reset_index()[['index','col1']].itertuples(index=False))

,3,2,0,1
0,2003,1,1,1
1,2004,1,1,1
2,2005,1,1,1
3,2006,1,1,1
4,2007,1,1,1
5,2008,1,1,1
6,2009,1,1,1
7,2010,1,1,1
8,2011,1,1,1
9,2012,1,1,1


In [ ]:
for group in eventTypes:
    yy = xx[xx[2] == group].drop([1,2],1)
    list(yy.itertuples(index=False))

In [ ]:
xx

In [87]:
yy = xx[xx[2] == 'roadwork'].drop([1,2],1)
list(yy.itertuples(index=False))

[Pandas(_0='2008', _1=1),
 Pandas(_0='2009', _1=4),
 Pandas(_0='2010', _1=32),
 Pandas(_0='2011', _1=1),
 Pandas(_0='2012', _1=8),
 Pandas(_0='2013', _1=2),
 Pandas(_0='2014', _1=13)]

In [69]:
xx = pd.DataFrame(mypoint[pos]).groupby([2, 3]).count().reset_index()
xx

,2,3,0,1
0,accidentsAndIncidents,2003,1,1
1,accidentsAndIncidents,2004,4,4
2,accidentsAndIncidents,2005,5,5
3,accidentsAndIncidents,2006,1,1
4,accidentsAndIncidents,2007,3,3
5,accidentsAndIncidents,2008,40,40
6,accidentsAndIncidents,2009,83,83
7,accidentsAndIncidents,2010,145,145
8,accidentsAndIncidents,2011,149,149
9,accidentsAndIncidents,2012,119,119


In [211]:
def box_indices(row):
    x1, x2 = row['nw_lon'], row['se_lon']
    y1, y2 = row['se_lat'], row['nw_lat']

    ll = np.array([x1, y1])
    ur = np.array([x2, y2])
    
    for eventType in eventTypes:
        group = eventGroups.get_group(eventType)
        pts = group[['longitude', 'latitude']].as_matrix()
        inidx = np.all(np.logical_and(ll <= pts, pts <= ur), axis=1)
        inbox = pts[inidx]
        
        row[eventType] = len(inbox)

In [66]:
def accident_indices(row):
    x1, x2 = row['nw_lon'], row['se_lon']
    y1, y2 = row['se_lat'], row['nw_lat']

    ll = np.array([x1, y1])
    ur = np.array([x2, y2])
    
    group = eventGroups.get_group('accidentsAndIncidents')
    pts = group[['longitude', 'latitude']].as_matrix()
    inidx = np.all(np.logical_and(ll <= pts, pts <= ur), axis=1)
    inbox = pts[inidx]
    
    print len(inbox)

    return [inidx]

In [67]:
trialf = trials[0:5]
trialf.head()

,trial_id,nw_lat,nw_lon,se_lat,se_lon,start,end
0,p_00000001,39.182985,-76.798663,39.170238,-76.778150,2015-02-01T00:00:00-05:00,2015-03-01T00:00:00-05:00
1,p_00000002,39.303607,-76.829335,39.296709,-76.806935,2016-01-08T00:00:00-05:00,2016-01-22T00:00:00-05:00
2,p_00000003,38.946840,-76.783484,38.931581,-76.761971,2015-03-01T00:00:00-05:00,2015-04-01T00:00:00-05:00
3,p_00000004,38.996686,-77.159555,38.982995,-77.140682,2015-01-01T00:00:00-05:00,2015-02-01T00:00:00-05:00
4,p_00000005,39.283366,-76.562381,39.265534,-76.544242,2016-04-05T00:00:00-05:00,2016-04-14T00:00:00-05:00


In [68]:
trialf.head()

,trial_id,nw_lat,nw_lon,se_lat,se_lon,start,end
0,p_00000001,39.182985,-76.798663,39.170238,-76.778150,2015-02-01T00:00:00-05:00,2015-03-01T00:00:00-05:00
1,p_00000002,39.303607,-76.829335,39.296709,-76.806935,2016-01-08T00:00:00-05:00,2016-01-22T00:00:00-05:00
2,p_00000003,38.946840,-76.783484,38.931581,-76.761971,2015-03-01T00:00:00-05:00,2015-04-01T00:00:00-05:00
3,p_00000004,38.996686,-77.159555,38.982995,-77.140682,2015-01-01T00:00:00-05:00,2015-02-01T00:00:00-05:00
4,p_00000005,39.283366,-76.562381,39.265534,-76.544242,2016-04-05T00:00:00-05:00,2016-04-14T00:00:00-05:00


In [69]:
trialf['accidents'] = trialf.apply(lambda row: accident_indices(row), axis=1)

1971
74
882
1783
2182


C:\Users\Ashok\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [60]:
start_time = time.time()
pts = mypoint

for tin in range(5):

    x1, x2 = trials.loc[tin]['nw_lon'], trials.loc[tin]['se_lon']
    y1, y2 = trials.loc[tin]['se_lat'], trials.loc[tin]['nw_lat']


    ll = np.array([x1, y1])  # lower-left
    ur = np.array([x2, y2])  # upper-right

    inidx = np.all(np.logical_and(ll <= pts, pts <= ur), axis=1)
    inbox = pts[inidx]
    #outbox = pts[np.logical_not(inidx)]
    #return len(inbox)
    
    print len(inbox)
    #pos = np.where(inidx==True)
    #print pos

NameError: name 'mypoint' is not defined

In [165]:
train = pd.read_csv("training.csv",header=0)

In [64]:
trials[1] = 0
trials[2] = 0
trials[3] = 0
trials[4] = 0
trials[5] = 0
trials[6] = 0


In [68]:
trials.head()

,trial_id,nw_lat,nw_lon,se_lat,se_lon,start,end,0,"(0, 1)",1,2,3,4,5,6
0,p_00000001,39.182985,-76.798663,39.170238,-76.778150,2015-02-01T00:00:00-05:00,2015-03-01T00:00:00-05:00,0,0,0,0,0,0,0,0
1,p_00000002,39.303607,-76.829335,39.296709,-76.806935,2016-01-08T00:00:00-05:00,2016-01-22T00:00:00-05:00,0,0,0,0,0,0,0,0
2,p_00000003,38.946840,-76.783484,38.931581,-76.761971,2015-03-01T00:00:00-05:00,2015-04-01T00:00:00-05:00,0,0,0,0,0,0,0,0
3,p_00000004,38.996686,-77.159555,38.982995,-77.140682,2015-01-01T00:00:00-05:00,2015-02-01T00:00:00-05:00,0,0,0,0,0,0,0,0
4,p_00000005,39.283366,-76.562381,39.265534,-76.544242,2016-04-05T00:00:00-05:00,2016-04-14T00:00:00-05:00,0,0,0,0,0,0,0,0
